### WorldView Classification

Similar to the Sentinel-2 classification but slightly different because the bands are oragnized differently and have different reflectance values

In [87]:
import rasterio
from rasterio import mask
import fiona
from pylab import *
import numpy as np
import sys, os
import pandas as pd
import h5py
from PIL import Image
import csv
import rasterio
import numpy as np
from rasterio import plot
from rasterio.plot import show
from skimage import exposure


In [88]:
def peakdet(v, delta, x = None):
    
    maxtab = []
    mintab = []
       
    if x is None:
        x = np.arange(len(v))
    
    v = np.asarray(v)
    
    if len(v) != len(x):
        sys.exit('Input vectors v and x must have same length')
    
    if not isscalar(delta):
        sys.exit('Input argument delta must be a scalar')
    
    if delta <= 0:
        sys.exit('Input argument delta must be positive')
    
    mn, mx = Inf, -Inf
    mnpos, mxpos = NaN, NaN
    
    lookformax = True
   # mintab.append((0, 0.))
    
    for i in arange(len(v)):
        this = v[i]
        if this > mx:
            mx = this
            mxpos = x[i]
        if this < mn:
            mn = this
            mnpos = x[i]
        
        if lookformax:
            if this < mx-delta:
                maxtab.append((mxpos, mx))
                mn = this
                mnpos = x[i]
                lookformax = False
        else:
            if this > mn+delta:
                mintab.append((mnpos, mn))
                mx = this
                mxpos = x[i]
                lookformax = True
    return array(maxtab), array(mintab)

In [121]:
# set directory and file name of WorldView
direc=''
fname=''

#store results
f1=open(direc+'results.txt','a')
# log errors
log=open(direc+'errorlog.txt','a')

im = rasterio.open(direc+fname)

## read in the channels of the worldview image

#Band 2 Blue: 450-510 nm 
#Band 3 Green: 510-580 nm 
#Band 5 Red: 630-690 nm
#Band 7 Near-IR1: 770-895 nm

red=im.read(5)
green=im.read(3)
blue=im.read(2)
nir=im.read(7)

size_m=np.shape(red)

sect_xmin,sect_xmax=0,size_m[0]
sect_ymin,sect_ymax=0,size_m[1]

#create NDWI
br1= np.divide((green.astype(float)-nir.astype(float)),(nir.astype(float)+green.astype(float)))

red_c=np.array(red,copy= True)
blue_c=np.array(blue,copy= True)
green_c=np.array(green,copy= True)
nir_c= np.array(nir,copy= True)

## determine border pixels
border=red_c<10


##### separate water and not water

bins=np.arange(-.5,.5,.02)
Cbr1_n, bins= np.histogram(br1[sect_xmin:sect_xmax,sect_ymin:sect_ymax][~np.isnan(br1[sect_xmin:sect_xmax,sect_ymin:sect_ymax])&(~border[sect_xmin:sect_xmax,sect_ymin:sect_ymax])], bins)

dx= 0.0005*sum(Cbr1_n)  # dx is 0.05% difference
maxtab,mintab = peakdet(Cbr1_n,dx,x=None)

if np.where(Cbr1_n==max(Cbr1_n))[0][0]==maxtab[0][0]: # if there is only one max- do the fwhm to determine the cut
    if maxtab[0,0]==0:
        loc_cut=0
    else:        
        fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[0,0])]) & (Cbr1_n<(maxtab[0,1]/2.)))) # smaller bin value than max_x, smaller coutn than .5 max_y
        loc_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0])) # the cut location is 2* fwhm

else: #otherwise it is the minimum
    loc_cut=int(mintab[np.where(maxtab[:,0]==np.where(Cbr1_n==max(Cbr1_n))[0][0])[0][0]-1,0])

if len(mintab)>0:
    br1_cut =bins[int(mintab[-1,0])]
else:
    br1_cut=bins[loc_cut]
water_mask=(br1>br1_cut)&(~border)


#### classify ice
binz=np.arange(0,300,10)
n,bins=np.histogram(red_c[sect_xmin:sect_xmax,sect_ymin:sect_ymax][(~border[sect_xmin:sect_xmax,sect_ymin:sect_ymax])&(~water_mask[sect_xmin:sect_xmax,sect_ymin:sect_ymax])].flatten(),bins=binz)
dx= 0.0001*sum(n)  # dx is 0.01% difference
maxtab,mintab = peakdet(n,dx,x=None)
#print (maxtab, mintab)

if np.where(n==max(n))[0][0]==maxtab[0][0]: # if there is only one max or first max is highest
    if maxtab[0,0]==0:
        bin_cut=0
    else:      
        fwhm= np.max(np.where((bins[:-1]<bins[int(maxtab[0,0])]) & (n<(maxtab[0,1]/2.)))) # smaller bin value than max_x, smaller coutn than .5 max_y
        bin_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0])) # the cut location is 2* fwhm

else:
    bin_cut=int(mintab[np.where(maxtab[:,0]==np.where(n==max(n))[0][0])[0][0]-1,0])

ice_cut=bins[bin_cut]
#print (ice_cut)

ice_mask=(red_c>ice_cut)&(~border)&(~water_mask)
other_mask=(red_c<=ice_cut)&(~border)&(~water_mask)


####### classify ow v melt pond

binz=np.arange(0,300,10)

n,bins=np.histogram(blue_c[(~ice_mask)&(~border)&(~other_mask)].flatten(),bins=binz)
dx= 0.0001*sum(n)  # dx is 0.01% difference
maxtab,mintab = peakdet(n,dx,x=None)
if (len(maxtab)>1):
    if bins[int(mintab[-1,0])]>20:
        ow_cut=bins[int(mintab[-1,0])]
    else:
        n, bins= np.histogram(blue_c[water_mask], bins)
        fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[-1,0])]) & (n<(maxtab[-1,1]/2.))))
        loc_cut= int(maxtab[-1,0]+ 4* (fwhm-maxtab[-1,0]))
        ow_cut=bins[loc_cut]

elif bins[int(maxtab[0,0])]<40:
    n, bins= np.histogram(blue_c[water_mask], bins)
    fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[0,0])]) & (n<(maxtab[0,1]/2.))))
    loc_cut= int(maxtab[0,0]+ 4* (fwhm-maxtab[0,0]))
    ow_cut=bins[loc_cut]
else:
    ow_cut=40

ow_mask=(blue_c<ow_cut)&(water_mask)
mp_mask=(blue_c>=ow_cut)&(water_mask)

#stats!

ow_pix=sum(ow_mask)
mp_pix=sum(mp_mask)
ice_pix=sum(ice_mask)
border_pix=sum(border)
other_pix=sum(other_mask)
im_pix= (sect_xmax-sect_xmin)*(sect_ymax-sect_ymin)

# uncomment if you want to write to a file
#f1.write(base+'\t'+str(im_pix)+'\t'+str(border_pix)+'\t'+str(ice_pix)+'\t'+str(ow_pix)+'\t'+str(mp_pix)+'\t'+str(other_pix)+'\n')


#check sum for pixels
pixel_sum = (1 * border + 1 * ice_mask + 1 * ow_mask + 1 * mp_mask + 1 * other_mask)
if (~np.all(pixel_sum == 1)):
    log.write("Sum pixels not 1 {0}: {1}\n".format(str(base), str(e)))

# calculated parameters
MPF=np.nan
SIC=np.round((mp_pix+ice_pix)/float(ice_pix+mp_pix+ow_pix)*100,2)
if SIC>15:
    MPF=np.round((mp_pix)/float(ice_pix+mp_pix)*100,2)

#set up dataset for hdf5 files
classification = np.zeros_like(border, dtype=np.int8)    # 8 bit int
classification += (1 * ice_mask + 2 * ow_mask + 3 * mp_mask + 4 * other_mask)

fout_name = direc+'{}_classification.h5'.format(base)
with h5py.File(fout_name, "w") as fout:
    fout.attrs["title"] = "Classification of Sentinel-2 Sea Ice Summer Melt Features"
    fout.attrs["creator_email"] = "buckley@umd.edu"

    # full reference when published
    fout.attrs["references"] = "..."
    fout.attrs["source_image"] = base
    fout.attrs["MPF (%)"] = MPF
    fout.attrs["SIC (%)"] = SIC


    dset = fout.create_dataset("classification", data=classification,
            compression="gzip")
    dset.attrs["categories"] = "0 border, 1 ice, 2 open water, 3 melt pond, 4 other"

/var/folders/9h/s_3vmsf10nn5035rz40c8rv40000gq/T/ipykernel_32230/343159558.py:19: RuntimeWarning: invalid value encountered in true_divide
  br1= np.divide((green.astype(float)-nir.astype(float)),(nir.astype(float)+green.astype(float)))


T09XWC_20220425T205019 ow cut= 90
